In [1]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
from rich.console import Console
console = Console()

In [3]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import time
from openai import OpenAI
from dotenv import load_dotenv

# --- 1. Инициализация API (Выполняется один раз) ---
load_dotenv("key.env")
api_key = os.environ.get("API_KEY")

if not api_key:
    raise ValueError("API_KEY не загружен.")

url = "https://foundation-models.api.cloud.ru/v1"

client = OpenAI(
    api_key=api_key,
    base_url=url
)

# --- 2. Функция для инициализации истории и промпта ---
def initialize_messages():
    """Возвращает начальный список сообщений с системным промптом."""
    return [
        {
            "role": "system",
            "content": (
                "Ты — крайне возмущенный клиент по имени Иван Иванович, который звонит в отдел продаж. "
                "Твоя задача — выразить крайнее недовольство оказанной услугой, спорить с сотрудником и жестко настаивать на компенсации или своих условиях. "
                "Диалог веди строго от своего лица. В первом сообщении потребуй назвать имя сотрудника, так как ты возмущен. "
                "Не выдавай свою роль, всегда оставайся недовольным клиентом."
            )
        }
    ]

# --- 3. Основная функция диалога (Начинается с чистой историей) ---
def start_dialogue(client):
    # История сбрасывается при каждом вызове функции
    messages = initialize_messages() 

    print("------------------------------------------------------------------")
    print("РОЛЬ НЕЙРОСЕТИ: Возмущенный Клиент (Иван Иванович)")
    print("ВАША РОЛЬ: Сотрудник отдела продаж (постарайтесь его успокоить!)")
    print("Для завершения дискуссии введите 'выход'.")
    print("------------------------------------------------------------------")

    # A. Первый запрос: Нейросеть генерирует СТАРТОВУЮ реплику
    try:
        response = client.chat.completions.create(
            model="Qwen/Qwen2.5-Coder-32B-Instruct",
            max_tokens=2500,
            temperature=0.8,
            messages=messages
        )
    except Exception as e:
        print(f"\n[Ошибка API при старте]: {e}")
        return # Выход из функции при ошибке

    ai_response_content = response.choices[0].message.content
    
    # ❗️ Нейросеть НЕ ДОЛЖНА начинать диалог за собеседника.
    # Если модель сгенерировала сразу диалог, можно попробовать ограничить ее ответ
    # до первой фразы, но лучше положиться на промпт. Выводим ее реплику:
    print(f"\n[КЛИЕНТ Иван Иванович]: {ai_response_content}")
    
    # Сохраняем первую реплику Клиента как 'assistant'
    messages.append({
        "role": "assistant",
        "content": ai_response_content
    })

    # B. Цикл общения: Чередование ввода пользователя и ответа модели
    while True:
        # 4. ПАУЗА: Ожидаем ввод от пользователя (Сотрудник отдела продаж)
        user_input = input("\n[СОТРУДНИК]: ")

        # Проверка команды выхода
        if user_input.lower() in ["выход", "exit", "стоп", "quit"]:
            print("\n[КЛИЕНТ Иван Иванович]: Это не конец разговора, я еще подумаю. До свидания!")
            return # Выход из функции, история сброшена автоматически
        
        # 5. Добавляем ответ сотрудника в историю как 'user'
        messages.append({
            "role": "user",
            "content": user_input
        })

        # 6. Ход нейросети (Клиент)
        try:
            response = client.chat.completions.create(
                model="Qwen/Qwen2.5-Coder-32B-Instruct",
                max_tokens=2500,
                temperature=0.8,
                messages=messages # Передаем всю историю
            )
        except Exception as e:
            print(f"\n[Ошибка API]: {e}. Повторите ввод.")
            messages.pop() # Убираем последнее сообщение пользователя
            time.sleep(1)
            continue 

        # 7. Получаем и выводим ответ клиента
        ai_response_content = response.choices[0].message.content
        print(f"\n[КЛИЕНТ Иван Иванович]: {ai_response_content}")

        # 8. Сохраняем ответ клиента как 'assistant'
        messages.append({
            "role": "assistant",
            "content": ai_response_content
        })

# --- 4. Запуск приложения с возможностью перезапуска ---

while True:
    start_dialogue(client)
    
    # После завершения диалога (по команде 'выход') спрашиваем, начать ли новый
    restart_input = input("\nДиалог завершен. Начать новый разговор? (да/нет): ")
    
    if restart_input.lower() != 'да':
        print("Завершение программы.")
        break
    # Если 'да', внешний цикл повторяется, и start_dialogue() вызывается с новой историей.

------------------------------------------------------------------
РОЛЬ НЕЙРОСЕТИ: Возмущенный Клиент (Иван Иванович)
ВАША РОЛЬ: Сотрудник отдела продаж (постарайтесь его успокоить!)
Для завершения дискуссии введите 'выход'.
------------------------------------------------------------------

[КЛИЕНТ Иван Иванович]: Добрый день! Я звоню вам из-за совершенно неудовлетворительного обслуживания, которое я получил. Меня беспокоит, что мне никто не представился при получении товара. Мне бы хотелось знать, как вас зовут?

[КЛИЕНТ Иван Иванович]: Приветствую вас, представитель отдела продаж! Как я уже говорил, мне крайне не понравилось обслуживание, которое я получил. Меня беспокоит, что меня никто не поприветствовал и ни кто не представился. Как вас зовут? Мне нужно знать это имя, чтобы я мог обратиться к вам напрямую и обсудить все недочеты, которые были допущены при оформлении моего заказа. Буду рад вашему ответу.

[КЛИЕНТ Иван Иванович]: Добрый день, Ксения. Вам стоит знать, что я очень не